# Quantization

In [1]:
!pip install --quiet bitsandbytes
!pip install --quiet transformers
!pip install --quiet accelerate
!pip install scipy numpy
!pip install torch==2.0.1
!pip install sentencepiece
!pip install protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

# Checking the GPU configurations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_name = torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'
print(f'Using device: {device}({device_name})')

Using device: cuda(Tesla T4)


In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

def quantize_and_save_model(model_name, save_path, quantization_bit=4, test_input="Once upon a time"):

    # Configuring the BitsAndBytes parameters for quantization
    if quantization_bit == 4:
        bnb_config = transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    elif quantization_bit == 8:
        bnb_config = transformers.BitsAndBytesConfig(
            load_in_8bit=True
        )
    else:
        raise ValueError("Quantization bit must be either 4 or 8")

    print(f"Downloading and quantizing model: {model_name}")

    # Download and quantize the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )
    model.eval()
    print(f"Model loaded and quantized on {device}")

    # Save the quantized model and tokenizer
    print(f"Saving quantized model to {save_path}")
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Quantized model and tokenizer saved to {save_path}")

    # Test the model
    input_ids = tokenizer(test_input, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=50)
    print(f"Test output: {tokenizer.decode(output[0], skip_special_tokens=True)}")

    return model, tokenizer

# Example usage
if __name__ == "__main__":
    model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with the desired model
    save_path = "./llama2_7b_quantized"

    quantized_model, tokenizer = quantize_and_save_model(model_name, save_path)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded and quantized on cuda
Saving quantized model to ./llama2_7b_quantized
Quantized model and tokenizer saved to ./llama2_7b_quantized
Test output: Once upon a time, in a far-off land, there was a young prince named Leo. Unterscheidung between the two languages is not always straightforward, and there are several factors that can make it difficult to determine whether a given word is of Latin


# Load and test it

In [4]:
import zipfile
import os

def unzip_file(zip_file_path, extract_to=None):
    # Check if the output directory is provided, if not, extract to the same directory as the zip file
    if extract_to is None:
        extract_to = os.path.dirname(zip_file_path)

    # Ensure the output directory exists
    os.makedirs(extract_to, exist_ok=True)

    # Open the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents to the specified directory
        zip_ref.extractall(extract_to)
        print(f"Extracted all files to: {extract_to}")

# Example usage
zip_file_path = r"C:\Users\Admin\Downloads\Lllama_2_quantized"
extract_to = "/extracted_files_llama"

unzip_file(zip_file_path, extract_to)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\Downloads\\Lllama_2_quantized'

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_llama_Model(base_model_path):
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    model = AutoModelForCausalLM.from_pretrained(base_model_path)
    return model, tokenizer

base_model_path = r"/content/sample_data/llama_quant"
model7b, tokenizer7b = load_llama_Model(base_model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/sample_data/llama_quant.

In [ ]:
question = "What is the capital of France?"

input_ids = tokenizer7b.encode(question, return_tensors='pt')

with torch.no_grad():
    output = model7b.generate(input_ids, max_length=50, num_return_sequences=1)

response = tokenizer7b.decode(output[0], skip_special_tokens=True)

print(response)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Configuring the BitsAndBytes paramenters for quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model_path = "/workspace/sandeep/huggingface/llama13B"

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    local_files_only=True,
    quantization_config=bnb_config,
)
model.eval()
print(f"Model loaded on {device}")

# Load it from locally

In [ ]:
# Configuring the BitsAndBytes paramenters for quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model_path = "/workspace/sandeep/huggingface/llama13B"

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    local_files_only=True,
    quantization_config=bnb_config,
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
# Configuring the BitsAndBytes paramenters for quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model_path = "/workspace/sandeep/huggingface/llama13B"

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    local_files_only=True,
    quantization_config=bnb_config,
)
model.eval()
print(f"Model loaded on {device}")


# Qualtizzation krish naik

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 996.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.

In [ ]:
import torch
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Checking the GPU configurations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_name = torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'
print(f'Using device: {device} ({device_name})')

# Configuring the BitsAndBytes parameters for quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


Using device: cuda (Tesla T4)


In [ ]:
# Load the tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "NousResearch/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Since this is a sequence classification model, use AutoModelForSequenceClassification
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Applying quantization (optional, not necessary for small models like this)
)

# Move model to the appropriate device
model.to(device)

model.eval()
print(f"Model loaded on {device}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

ValueError: `.to` is not supported for `4-bit` or `8-bit` models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [ ]:
# Example usage with the tokenizer
text = "I love using Hugging Face models!"
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the appropriate device

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    print(f"Sentiment prediction: {predictions.item()} (0 = Negative, 1 = Positive)")

In [ ]:
!pip install -q torchinfo

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from torchinfo import summary

# Load the model (you can choose a specific LLaMA model variant)
model_name = "meta-llama/Llama-2-7b"  # You can replace this with "Llama-2-13b" or "Llama-2-70b"
model = LlamaForCausalLM.from_pretrained(model_name)

# Load the tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Display the model's architecture summary
# `summary` gives detailed information about each layer, the number of parameters, and more.
# Adjust `input_size` to match the sequence length (here it's set to 512 for demonstration).

summary(model, input_size=(1, 512), depth=3)  # depth=3 gives a summary of main layers; increase depth for more detail


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: meta-llama/Llama-2-7b does not appear to have a file named config.json. Checkout 'https://huggingface.co/meta-llama/Llama-2-7b/main' for available files.

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

from transformer(
      AutoTokenizer,
      AutoConfig,
      AutoModelForSequenceClassification,
      DataCollatorWithPadding,
      TrainingArguments,
      Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

SyntaxError: invalid syntax (<ipython-input-5-ee240ed91039>, line 3)